### Предобработка текста

In [2]:
import pandas as pd

In [3]:
df = pd.read_csv("https://storage.yandexcloud.net/auth-def-2024/datasets/meta_table_with_texts.csv")
df = df[['author', 'text']]
df.head(5)

,author,text
0,Пушкин Александр Сергеевич,"\n \nЛитературный альбомъ.\n""Сраженный рыцар..."
1,Карамзин Николай Михайлович,\nО достоинстве древних и новых\n(Перевод с не...
2,Гоголь Николай Васильевич,\n Гоголь Н. В. Полное собрание сочинений и ...
3,Мамин-Сибиряк Дмитрий Наркисович,\n \nД. МАМИНЪ-СИБИРЯКЪПОЛНОЕ СОБРАНІЕ СОЧИН...
4,Мамин-Сибиряк Дмитрий Наркисович,\nДмитрий Мамин-Сибиряк\nНимфа\nI.\n Щегольс...


In [13]:
import re

# Очистка текста от незначащих символов
def clean_df(x):
  # Удаление unicode-символов
  x = re.sub(r'&#\d+;|&#x[0-9a-fA-F]+;', '', x)
  x = x.replace('\n', ' ')
  v2 = re.sub(r'[^А-Яа-я0-9\(\).,!?: \-]', '', x.lower())
  result = re.sub(r'\s+', ' ', v2).strip()
  return result

# Токенизация текста
def words_list(x):
  # Подсчёт слов в тексте
  words = re.findall(r'\b\w+\b', x)
  return [w.lower() for w in words]

In [16]:
df['clean_text'] = df['text'].apply(clean_df)

In [17]:
df['words'] = df['clean_text'].apply(words_list)
df.head(5)

,author,text,clean_text,words
0,Пушкин Александр Сергеевич,"\n \nЛитературный альбомъ.\n""Сраженный рыцар...",литературный альбомъ. сраженный рыцарь. пушкин...,"[литературный, альбомъ, сраженный, рыцарь, пуш..."
1,Карамзин Николай Михайлович,\nО достоинстве древних и новых\n(Перевод с не...,о достоинстве древних и новых (перевод с немец...,"[о, достоинстве, древних, и, новых, перевод, с..."
2,Гоголь Николай Васильевич,\n Гоголь Н. В. Полное собрание сочинений и ...,гоголь н. в. полное собрание сочинений и писем...,"[гоголь, н, в, полное, собрание, сочинений, и,..."
3,Мамин-Сибиряк Дмитрий Наркисович,\n \nД. МАМИНЪ-СИБИРЯКЪПОЛНОЕ СОБРАНІЕ СОЧИН...,д. маминъ-сибирякъполное собране сочиненйтомъ ...,"[д, маминъ, сибирякъполное, собране, сочиненйт..."
4,Мамин-Сибиряк Дмитрий Наркисович,\nДмитрий Мамин-Сибиряк\nНимфа\nI.\n Щегольс...,дмитрий мамин-сибиряк нимфа . щегольской волжс...,"[дмитрий, мамин, сибиряк, нимфа, щегольской, в..."


Загрузим эвристики, которые были рассчитаны и отобраны на этапе проведения экспериментов и замера метрик качества моделей.

In [44]:
# Загрузка фрейма с эвристиками
df_heuristics = pd.read_csv('heuristics.csv')
df_heuristics.head(5)

,avg_tonality,avg_subjectivity,total_words,avg_words_per_sentence
0,0.066585,0.229008,353,15.347826
1,-0.232829,0.358885,1008,15.750000
2,-0.010425,0.195767,196245,7.540634
3,-0.000427,0.007812,9247,8.353207
4,-0.000427,0.007812,4477,6.962675


### Формирование выборок

Обучение модели для дальнейшего формирования эмбеддингов:

In [22]:
from gensim.models import Word2Vec

w2v_model = Word2Vec(sentences=df['words'], vector_size=100, window=5, min_count=1, workers=4)

In [23]:
import numpy as np

# Функция для преобразования текста в усреднённый вектор эмбеддингов
def text_to_vector(text, model, vector_size=100):
    vectors = [model.wv[word] for word in text if word in model.wv]
    return np.mean(vectors, axis=0) if vectors else np.zeros(vector_size)

In [25]:
# Создание признаков эмбеддингов
df['embeddings'] = df['words'].apply(lambda x: text_to_vector(x, w2v_model))
df.head(5)

,author,text,clean_text,words,embeddings
0,Пушкин Александр Сергеевич,"\n \nЛитературный альбомъ.\n""Сраженный рыцар...",литературный альбомъ. сраженный рыцарь. пушкин...,"[литературный, альбомъ, сраженный, рыцарь, пуш...","[-0.110117756, 0.3974111, -0.28785634, 0.20738..."
1,Карамзин Николай Михайлович,\nО достоинстве древних и новых\n(Перевод с не...,о достоинстве древних и новых (перевод с немец...,"[о, достоинстве, древних, и, новых, перевод, с...","[-0.009144303, 1.1618452, -1.0102768, 1.41935,..."
2,Гоголь Николай Васильевич,\n Гоголь Н. В. Полное собрание сочинений и ...,гоголь н. в. полное собрание сочинений и писем...,"[гоголь, н, в, полное, собрание, сочинений, и,...","[0.49652144, 0.9374549, -1.3448213, 1.600433, ..."
3,Мамин-Сибиряк Дмитрий Наркисович,\n \nД. МАМИНЪ-СИБИРЯКЪПОЛНОЕ СОБРАНІЕ СОЧИН...,д. маминъ-сибирякъполное собране сочиненйтомъ ...,"[д, маминъ, сибирякъполное, собране, сочиненйт...","[0.007775612, 0.53899366, -0.44002172, 0.34412..."
4,Мамин-Сибиряк Дмитрий Наркисович,\nДмитрий Мамин-Сибиряк\nНимфа\nI.\n Щегольс...,дмитрий мамин-сибиряк нимфа . щегольской волжс...,"[дмитрий, мамин, сибиряк, нимфа, щегольской, в...","[0.43801323, 0.9110735, -1.2663436, 1.8236201,..."


На этапе проведения экспериментов было принято решение использовать комбинированный подход в выборе признаков.

In [29]:
# Разделение признаков и целевой переменной
X_embeddings = np.vstack(df['embeddings'].values)
X_heuristics = df_heuristics.values
y = df['author']

In [31]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Обучающая и тренировочная выборки
X_heur_train, X_heur_test, X_emb_train, X_emb_test, y_train, y_test = train_test_split(X_heuristics, X_embeddings, y, test_size=0.2, random_state=42)

# Стандартизация эвристик
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_heur_train)
X_test_scaled = scaler.transform(X_heur_test)

# Формирование итоговых датасетов
X_train_combined = np.hstack([X_train_scaled, X_emb_train])
X_test_combined = np.hstack([X_test_scaled, X_emb_test])

### Создание пайплайна

На этапе проведения экспериментов было выявлено, что лучшие метрики качества выдавала логистическая регрессия. Поэтому в пайплайне указываем `LogisticRegression`:

In [36]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression

# Пайплайн
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('classifier', LogisticRegression(max_iter=1000, random_state=42))
])

pipeline.fit(X_train_combined, y_train)
y_pred = pipeline.predict(X_test_combined)

In [42]:
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score, classification_report, confusion_matrix

# Оценка качества модели
# Accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

# Precision, Recall, F1 (взвешенные для многоклассовой задачи)
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')

print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)

# Матрица ошибок
conf_matrix = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:\n", conf_matrix)

Accuracy: 0.935672514619883
Precision: 0.9354854056749409
Recall: 0.935672514619883
F1 Score: 0.9350172853658607
Confusion Matrix:
 [[14  0  3  0  1  0  0  1  0  0  0  0  2]
 [ 2 45  0  0  0  0  1  0  2  0  0  0  0]
 [ 2  1 34  0  0  0  0  1  0  0  0  0  0]
 [ 0  0  0 42  0  0  0  0  0  1  0  2  0]
 [ 0  1  2  0 10  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0 59  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0 77  0  1  0  0  0  0]
 [ 0  1  0  0  0  0  0  6  0  0  0  0  0]
 [ 0  1  0  0  0  0  3  0 49  0  0  0  0]
 [ 0  0  1  0  0  0  0  0  0 27  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0 36  0  0]
 [ 0  0  0  1  0  0  0  0  0  0  0 60  0]
 [ 0  1  1  1  0  0  0  0  0  0  0  0 21]]


По итогу на комбинированном наборе фичей (эвристики + текстовые) логистическая регрессия показала значения метрик точности предсказания в районе $\approx 93\%$.